In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# Setup driver
driver = webdriver.Chrome()
driver.set_window_size(1300, 900)

# Link halaman Tokopedia
tokped_link = "https://www.tokopedia.com/search?st=&q=flat%20shoes&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&navsource="
driver.get(tokped_link)

data_produk = []

for page in range(3):  # Lakukan scraping pada 5 halaman
    time.sleep(3)

    # Scroll untuk memastikan semua produk di halaman terload
    for i in range(14):
        driver.execute_script(f"window.scrollTo(0, {500 * i});")
        time.sleep(1)

    # Parsing halaman setelah scroll
    data = BeautifulSoup(driver.page_source, 'html.parser')

    # Ambil data dari elemen produk
    for area in data.find_all('div', class_='css-5wh65g'):
        try:
            nama = area.find('div', class_="VKNwBTYQmj8+cxNrCQBD6g==").get_text()
            nama_toko = area.find('span', class_="X6c-fdwuofj6zGvLKVUaNQ== -9tiTbQgmU1vCjykywQqvA== flip").get_text()
            lokasi = area.find('span', class_="-9tiTbQgmU1vCjykywQqvA== flip").get_text()
            harga = area.find('div', class_="ELhJqP-Bfiud3i5eBR8NWg==").get_text()
            terjual = area.find('span', class_="eLOomHl6J3IWAcdRU8M08A==")
            if terjual:
                terjual = terjual.get_text()
            rating = area.find('span', class_="nBBbPk9MrELbIUbobepKbQ==")
            if rating:
                rating = rating.get_text()
            gambar = area.find('img')['src']

            data_produk.append((nama, nama_toko, lokasi, harga, terjual, rating, gambar))
        except Exception as e:
            print("Error retrieving product data:", e)

    # Coba klik tombol 'Laman berikutnya' dengan explicit wait
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button[aria-label='Laman berikutnya']"))
        )
        driver.execute_script("arguments[0].scrollIntoView();", next_button)
        time.sleep(2)  # Tambahkan sedikit delay untuk memastikan elemen sudah stabil
        next_button.click()
        time.sleep(2)  # Tunggu halaman berikutnya untuk dimuat
    except Exception as e:
        print("Tidak dapat mengklik tombol 'Laman berikutnya':", e)
        break

# Simpan data ke file CSV/Excel
df = pd.DataFrame(data_produk, columns=['Nama', 'Nama toko', 'Lokasi', 'Harga', 'Terjual', 'Rating', 'Gambar'])
#df.to_excel('result_scaping_tokped_with_selenium.xlsx', index=False)
df.to_csv('result_scaping_tokped_with_selenium.csv', index=False)
print('Data telah tersimpan')

# Tutup driver
driver.quit()

Data telah tersimpan
